In [3]:
!python -V

Python 3.9.23


In [4]:
import pandas as pd
import numpy as np

In [5]:
import pickle

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [8]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/09/16 17:51:43 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/16 17:51:43 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1758017519199, experiment_id='1', last_update_time=1758017519199, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [9]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [10]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [11]:
len(df_train), len(df_val)

(73908, 61921)

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)


7.758715209663881

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Phan Dang Ha")
    
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)
    
    y_pred = lr.predict(X_val)
    # rmse = np.mean_squared_error(y_val, y_pred)
    rmse = root_mean_squared_error(y_val, y_pred)
    print(rmse)
    mlflow.log_metric("rmse", rmse)
    
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

12.143233939663894


In [18]:
import xgboost as xgb

In [19]:
xgb.__version__

'2.1.4'

In [20]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        # rmse = mean_squared_error(y_val, y_pred, squared=False)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    
    return {'loss': rmse, 'status': STATUS_OK}
print(rmse)

12.143233939663894


In [21]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,
    trials=Trials()
)

  0%|                                                   | 0/5 [00:00<?, ?trial/s, best loss=?]

/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:33:58] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.99053                                                                  
[1]	validation-rmse:10.01117                                                                  
[2]	validation-rmse:9.23275                                                                   
[3]	validation-rmse:8.62058                                                                   
[4]	validation-rmse:8.14359                                                                   
[5]	validation-rmse:7.77367                                                                   
[6]	validation-rmse:7.48795                                                                   
[7]	validation-rmse:7.26933                                                                   
[8]	validation-rmse:7.10247                                                                   
[9]	validation-rmse:6.97356                                                                   
[10]	validation-rmse:6.87447                      

/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:36:58] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.38513                                                                  
[1]	validation-rmse:9.11024                                                                   
[2]	validation-rmse:8.23718                                                                   
[3]	validation-rmse:7.65969                                                                   
[4]	validation-rmse:7.27956                                                                   
[5]	validation-rmse:7.02613                                                                   
[6]	validation-rmse:6.85962                                                                   
[7]	validation-rmse:6.74933                                                                   
[8]	validation-rmse:6.67218                                                                   
[9]	validation-rmse:6.61856                                                                   
[10]	validation-rmse:6.57855                      

...

[0]	validation-rmse:7.82778                                                                   
[1]	validation-rmse:6.95746                                                                   
[2]	validation-rmse:6.78049                                                                   
[3]	validation-rmse:6.72575                                                                   
[4]	validation-rmse:6.70755                                                                   
[5]	validation-rmse:6.69977                                                                   
[6]	validation-rmse:6.69444                                                                   
[7]	validation-rmse:6.68970                                                                   
[8]	validation-rmse:6.68649                                                                   
[9]	validation-rmse:6.68435                                                                   
[10]	validation-rmse:6.68128                      

/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:39:04] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.54722                                                                   
[1]	validation-rmse:7.19580                                                                   
[2]	validation-rmse:6.74180                                                                   
[3]	validation-rmse:6.58242                                                                   
[4]	validation-rmse:6.50911                                                                   
[5]	validation-rmse:6.47492                                                                   
[6]	validation-rmse:6.45698                                                                   
[7]	validation-rmse:6.44771                                                                   
[8]	validation-rmse:6.43948                                                                   
[9]	validation-rmse:6.43687                                                                   
[10]	validation-rmse:6.43090                      

/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [16:40:01] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.14513                                                                  
[1]	validation-rmse:10.25723                                                                  
[2]	validation-rmse:9.52732                                                                   
[3]	validation-rmse:8.93087                                                                   
[4]	validation-rmse:8.44391                                                                   
[5]	validation-rmse:8.05214                                                                   
[6]	validation-rmse:7.73976                                                                   
[7]	validation-rmse:7.48780                                                                   
[8]	validation-rmse:7.28557                                                                   
[9]	validation-rmse:7.12476                                                                   
[10]	validation-rmse:6.99552                      

In [22]:
params = {
    'learning_rate': 0.20472169880371677,
    'max_depth': 17,
    'min_child_weight': 1.2402611720043835,
    'objective': 'reg:linear',
    'reg_alpha': 0.28567896734700793,
    'reg_lambda': 0.004264404814393109,
    'seed': 42 
 
}
    
with mlflow.start_run():
    mlflow.set_tag('model', 'xgboost')
    mlflow.log_params(params)
    booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
    )
 
    


/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:12:31] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

In [23]:
params = {
    'learning_rate': 0.20472169880371677,
    'max_depth': 17,
    'min_child_weight': 1.2402611720043835,
    'objective': 'reg:linear',
    'reg_alpha': 0.28567896734700793,
    'reg_lambda': 0.004264404814393109,
    'seed': 42 
}
    
mlflow.xgboost.autolog()
    # mlflow.set_tag("model", "xgboost")
    # mlflow.log_params(params)
booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2025/09/16 17:21:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '23dac403502644e885367e5896c95d97', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:21:20] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

2025/09/16 17:23:06 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/09/16 17:23:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/16 17:23:06 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:23:06] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/09/16 17:23:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [23]:
with mlflow.start_run():

# mlflow.xgboost.autolog()  # bật tự động log

    best_params = {
        'learning_rate': 0.20472169880371677,
        'max_depth': 17,
        'min_child_weight': 1.2402611720043835,
        'objective': 'reg:linear',
        'reg_alpha': 0.28567896734700793,
        'reg_lambda': 0.004264404814393109,
        'seed': 42 
    }

    mlflow.log_params(best_params)
    
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow' )
    # print("Validation RMSE:", rmse)


/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:52:23] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

2025/09/16 17:54:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/opt/conda/envs/mlops-zoomcamp/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [17:54:01] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/09/16 17:54:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
